In [30]:
import os
import ipyvuetify as v
import scrapbook as sb
import yaml
import shutil

In [2]:
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)

In [3]:
title_bar_color = config['title_bar_color']

In [4]:
logo = {
    'jupyter': 'https://jupyter.org/assets/nav_logo.svg'
}

In [5]:
my_dashboard_name = config['title']
voila_base_url = config['server_url'] + '/voila/render/'

In [6]:
additional = []
if 'additional' in config.keys():
    additional = [dictionary for key, dictionary in config['additional'].items()]

In [7]:
lst = []
if 'app_bar_links' in config.keys():
    lst = [
        v.Btn(children=[name], flat=True, href=link, target='_blank')
        for name, link in config['app_bar_links'].items()
    ]

In [8]:
all_files = [f for f in os.listdir('.') if os.path.isfile(f)]
all_files.remove('homepage.ipynb')

In [9]:
files = []
for file in all_files:
    extension = file.split('.')[-1]
    if extension == 'ipynb':
        files.append(file)

In [10]:
items = []
for i, f in enumerate(files):
    this_item = {}
    this_item['title'] = f
    this_item['description'] = 'A Jupyter Notebook.'
    this_item['link'] = voila_base_url + f
    this_item['logo'] = 'https://jupyter.org/assets/nav_logo.svg'
    this_item['fname'] = f

    nb = sb.read_notebook(f)
    for key, value in nb.scraps.data_dict.items():
        this_item[key] = value
    items.append(this_item)

In [11]:
items.extend(additional)

In [12]:
toolbar = v.Toolbar(color=title_bar_color, dark=True,
    children=[
        v.ToolbarItems(children=[v.Img(src=logo['jupyter'], style_='height:100%')]),
        v.ToolbarTitle(children=[my_dashboard_name], color='green'),
        v.Spacer(), # Fills empty space
        v.ToolbarItems(children=lst)
    ],
    app=True
)

In [38]:
dels = [
    v.Btn(icon=True, href='./homepage.ipynb', children=[v.Icon(color='grey', children=['delete'])])
    #v.Btn(icon=True, href='', children=[v.Icon(color='grey', children=['delete'])])
    for i, details in enumerate(items)
]

def on_del_click(widget, event, data):
    fname = (items[dels.index(widget)]['fname'])
    if not os.path.exists('./deleted'):
        os.makedirs('./deleted')
    shutil.move('./'+fname, './deleted/'+fname)

for button in dels:
    button.on_event('click', on_del_click)

In [39]:
cards = [
    v.Flex(ma_2=True, fluid=True, children = [
        v.Card(hover=True, #class_='mx-auto', elevation=5,
               align_center=True,
               fluid=True,
               #height='100%',
               #min_height='250px',
               min_width='300px',
               max_width='300px',
               href=details['link'],
               target='_blank',
               children=[
                   v.CardTitle(children=[
                       v.Html(tag='div', class_='headline mb-0', children=[details['title']]),
                       v.Spacer(),
                       #v.Btn(icon=True, href=details['link'], target='blank_', children=[v.Icon(color='grey', children=['launch'])])
                   ]),
                   v.CardText(children=[details['description']]),
                   v.CardActions(ma_3=True, children=[
                       dels[i],
                       v.Spacer(),
                       v.Img(src=details['logo'], max_width=80, contain=True)])
               ])
    ])
for i, details in enumerate(items)
]

In [40]:
app = v.Container(
    fluid = True,
    children = [
        toolbar,
        v.Layout(ma_5=True, wrap=True, children=cards),
    ]
)

In [41]:
app

Container(children=[Toolbar(app=True, children=[ToolbarItems(children=[Img(src='https://jupyter.org/assets/nav…

In [17]:
    # delete = v.Btn(color='warn', children=['delete'])
    # cancel = v.Btn(children=['cancel'])
    # def close_dialog(a,b,c):
    #     dlg.v_model=False
    # cancel.on_event('click', close_dialog)

In [18]:

# items = [v.ListTile(children=[
#     v.ListTileTitle(children=[
#         f'Click me {i}'])]) 
#          for i in range(1, 5)]
# dlst = v.List(children=items)

In [19]:
??v.Footer


In [20]:
# img=v.Img(class_='ma-3', src='https://jupyter.org/assets/nav_logo.svg', max_width=80, contain=True)

In [21]:
# z = v.Card(class_='mx-auto', elevation=5,
#                align_center=True,
#                min_height='300px',
#                min_width='400px',
#                max_height='300px',
#                max_width='400px',
#                children=[
#                    v.CardTitle(children=[v.Html(tag='div', class_='headline mb-0', children=['title'])]),
#                    v.CardText(children=['description']),
#                    v.Flex(children=[v.Footer(class_='ma-3', children=[img], absolute=True)]),
#                ])

In [22]:
#dlst

In [23]:
# dlg = v.Dialog(v_model=False, width='500', children=[
#         v.Btn(icon=True, slot='activator', small=True, children=[v.Icon(color='grey', children=['delete'])]),
#         v.Card(children=[
#             v.CardTitle(children=['Delete Notebook']),
#             v.CardMedia(children=[
#                dlst
#             ]),
#             v.CardActions(children=[cancel, delete])
#         ])
# ])